# Servidor

In [2]:
import socket
import threading
import os
import ssl

def start_server():
    HOST = '127.0.0.1'
    PORT = 65432

    # Criação do contexto SSL
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile="cert.pem", keyfile="key.pem")  # Certificado SSL

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((HOST, PORT))
    server_socket.listen()
    print(f"Servidor rodando em {HOST}:{PORT}")

    # Wrap no socket com o contexto SSL
    server_socket = context.wrap_socket(server_socket, server_side=True)

    def handle_client(conn, addr):
        print(f"Conexão estabelecida com {addr}")
        try:
            while True:
                command = conn.recv(1024).decode()
                if not command:
                    break
                elif command == "LIST":
                    files = os.listdir(".")
                    conn.send("\n".join(files).encode())
                elif command.startswith("UPLOAD"):
                    _, filename = command.split()
                    with open(filename, "wb") as f:
                        data = conn.recv(1024)
                        while data:
                            f.write(data)
                            data = conn.recv(1024)
                    print(f"Arquivo {filename} recebido.")
                elif command.startswith("DOWNLOAD"):
                    _, filename = command.split()
                    if os.path.exists(filename):
                        conn.send(f"OK {os.path.getsize(filename)}".encode())
                        with open(filename, "rb") as f:
                            conn.sendfile(f)
                    else:
                        conn.send("ERRO: Arquivo não encontrado.".encode())
                elif command == "QUIT":
                    print(f"Cliente {addr} desconectou.")
                    break
        except Exception as e:
            print(f"Erro ao lidar com cliente {addr}: {e}")
        finally:
            conn.close()

    try:
        while True:
            conn, addr = server_socket.accept()
            threading.Thread(target=handle_client, args=(conn, addr)).start()
    except Exception as e:
        print(f"Erro inesperado no servidor: {e}")
    finally:
        server_socket.close()

# Execute o servidor em uma thread
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()


Servidor rodando em 127.0.0.1:65432


# cliente

In [ ]:
# import socket
import ssl
import os
from IPython.display import clear_output

HOST = '127.0.0.1'
PORT = 65432

# Criação do contexto SSL
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
context.load_verify_locations(cafile="cert.pem")  # Certificado do servidor

try:
    # Criação de um socket seguro
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket = context.wrap_socket(client_socket, server_hostname=HOST)
    client_socket.connect((HOST, PORT))

    print("Conectado ao servidor de forma segura.")

    while True:
        # Limpa a tela para deixar as opções mais organizadas
        clear_output(wait=True)
        
        print("Opções:")
        print("1. Listar arquivos")
        print("2. Enviar arquivo")
        print("3. Baixar arquivo")
        print("4. Sair")
        option = input("Escolha uma opção: ").strip()  # Remove espaços extras

        if not option:  # Ignora entradas vazias
            print("Entrada inválida. Tente novamente.")
            continue
        
        if option == "1":
            try:
                client_socket.send("LIST".encode())
                files = client_socket.recv(1024).decode()
                print("Arquivos no servidor:")
                print(files)
            except Exception as e:
                print(f"Erro ao listar arquivos: {e}")
            input("Pressione Enter para continuar...")  # Pausa antes de limpar a tela
        elif option == "2":
            filename = input("Digite o nome do arquivo para enviar: ").strip()
            try:
                if not filename:  # Ignora entradas vazias
                    print("Nome do arquivo inválido. Tente novamente.")
                    continue
                
                # Verifica se o arquivo existe e o tamanho
                if not os.path.exists(filename):
                    print("Arquivo não encontrado.")
                    continue
                if os.path.getsize(filename) > 10 * 1024 * 1024:  # Limite de 10 MB
                    print("Arquivo excede o tamanho máximo permitido (10 MB).")
                    continue

                with open(filename, "rb") as f:
                    client_socket.send(f"UPLOAD {filename}".encode())
                    client_socket.sendfile(f)
                print(f"Arquivo {filename} enviado com sucesso.")
            except Exception as e:
                print(f"Erro ao enviar arquivo: {e}")
            input("Pressione Enter para continuar...")  # Pausa antes de limpar a tela
        elif option == "3":
            filename = input("Digite o nome do arquivo para baixar: ").strip()
            try:
                if not filename:  # Ignora entradas vazias
                    print("Nome do arquivo inválido. Tente novamente.")
                    continue
                
                client_socket.send(f"DOWNLOAD {filename}".encode())
                response = client_socket.recv(1024).decode()
                if response.startswith("OK"):
                    size = int(response.split()[1])
                    if size > 10 * 1024 * 1024:  # Limite de 10 MB
                        print("Arquivo excede o tamanho máximo permitido (10 MB).")
                        continue

                    with open(f"baixado_{filename}", "wb") as f:
                        data = client_socket.recv(size)
                        f.write(data)
                    print(f"Arquivo {filename} baixado com sucesso.")
                else:
                    print(response)
            except Exception as e:
                print(f"Erro ao baixar arquivo: {e}")
            input("Pressione Enter para continuar...")  # Pausa antes de limpar a tela
        elif option == "4":
            try:
                client_socket.send("QUIT".encode())
                print("Desconectado do servidor.")
                break
            except Exception as e:
                print(f"Erro ao desconectar: {e}")
        else:
            print("Opção inválida. Tente novamente.")
            input("Pressione Enter para continuar...")  # Pausa antes de limpar a tela
except Exception as e:
    print(f"Erro ao conectar ou executar o cliente: {e}")
finally:
    if 'client_socket' in locals():
        client_socket.close()
    print("Conexão encerrada.")


Conectado ao servidor.

Opções:
1. Listar arquivos
2. Enviar arquivo
3. Baixar arquivo
4. Sair
